In [1]:
!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [1]:
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [23]:
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [4]:
# laoding the document
loader = UnstructuredFileLoader("Const.pdf")
documents = loader.load()

In [5]:
# splitting into text chunks
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

In [6]:
texts = text_splitter.split_documents(documents)

In [7]:
type(texts)

list

In [8]:
len(texts)

212

In [ ]:
#texts[:3]

In [9]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-9-d0c9174021d8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
persist_directory = "doc_db"

In [11]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [12]:
# retriever
retriever = vectordb.as_retriever()

In [13]:
# llm from groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [14]:
# create a qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [18]:
# invoke the qa chain and get a response for user query
query = "How foreign women can get citizenship?"
response = qa_chain.invoke({"query": query})

In [19]:
print(response)

{'query': 'How foreign women can get citizenship?', 'result': 'According to the provided context, a foreign woman who has a matrimonial relationship with a citizen of Nepal may acquire the naturalized citizenship of Nepal as provided for in the Federal law, if she so wishes.', 'source_documents': [Document(metadata={'source': 'Const.pdf'}, page_content="(4) Every minor who is found within Nepal and the whereabouts of whose father and mother are not known shall, until the father or the mother of the child is traced, be a citizen of Nepal by descent.\n\n(5) A person who is born in Nepal from a woman who is a citizen of Nepal and has resided in Nepal and\n\n(6)\n\nwhose father is not traced shall be provided with the citizenship of Nepal by descent.\n\nProvided that his or her father is held to be a foreign citizen, the citizenship of such person shall be converted into naturalized citizenship as provided for in the Federal law.\n\n(6) A foreign woman who has a matrimonial relationship wi

In [20]:
print(response["result"])

According to the provided context, a foreign woman who has a matrimonial relationship with a citizen of Nepal may acquire the naturalized citizenship of Nepal as provided for in the Federal law, if she so wishes.


In [21]:
print(response["source_documents"][0].metadata["source"])

Const.pdf


In [22]:
# invoke the qa chain and get a response for user query
query = "How foreign women can get citizenship?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

According to the provided context, a foreign woman who has a matrimonial relationship with a citizen of Nepal may acquire the naturalized citizenship of Nepal as provided for in the Federal law, if she so wishes.
******************************
Source Document: Const.pdf
